# Example 6: Orbit Propagation with J2 Perturbation


## Problem Statement
This example demonstrates how to propagate a spacecraft's orbit while accounting for the perturbing effect of Earth's oblateness, modeled by the J2 zonal harmonic.

Unlike the classical two-body model (which assumes Earth is perfectly spherical), the J2 perturbation introduces long-term effects:
- **Nodal regression** (westward drift of the orbital plane).
- **Apsidal rotation** (rotation of the line of apsides).

### Initial Conditions:
- Position vector (km): $r_0 = [840.5, 485.3, 6905.8]$
- Velocity vector (km/s): $v_0 = [3.7821, -6.5491, 0.0057]$


### Governing Equations
The two-body acceleration is given by:

$$
\ddot{\mathbf{r}}_{2body} = -\mu \frac{\mathbf{r}}{\lVert \mathbf{r} \rVert^3}
$$

where $\mu = 398600.4418\,\text{km}^3/\text{s}^2$ is Earth's gravitational parameter.

The J2 perturbation acceleration components are:

$$
\ddot{\mathbf{r}}_{J2} = -\mu \frac{\mathbf{r}}{r^3} \begin{bmatrix}
1 + \tfrac{3}{2} J_2 \left(\tfrac{R_E}{r}\right)^2 \left(1 - 5\tfrac{z^2}{r^2}\right) \\
1 + \tfrac{3}{2} J_2 \left(\tfrac{R_E}{r}\right)^2 \left(1 - 5\tfrac{z^2}{r^2}\right) \\
1 + \tfrac{3}{2} J_2 \left(\tfrac{R_E}{r}\right)^2 \left(3 - 5\tfrac{z^2}{r^2}\right)
\end{bmatrix}
$$

where:
- $R_E = 6378.137$ km is Earth's equatorial radius
- $J_2 = 1.08263 \times 10^{-3}$ is the second zonal harmonic coefficient
- $r = \sqrt{x^2+y^2+z^2}$

The total acceleration is then:

$$
\ddot{\mathbf{r}} = \ddot{\mathbf{r}}_{2body} + \ddot{\mathbf{r}}_{J2}
$$


### Method:
We use the `Orbit_2body` class (from `orbit_util.py`), which includes a `propagate_with_J2()` method.
The orbit is propagated over **10 days** ($864,000$ seconds) with a $10$-second timestep.

The `OrbitVisualizer` class is used to generate both static and animated orbit visualizations.

### Additional Capabilities:
This notebook also demonstrates:
- Time conversions (UTC → Julian Date, YYDDD, formatted UTC strings).
- Exporting ephemeris files compatible with FreeFlyer/STK (`.e`) and SPICE (`.bsp`).


### Code

Here is the example code:

In [ ]:
import numpy as np
from datetime import datetime, timezone
from orbit_util import Orbit_2body, OrbitVisualizer

# -------------------------
# Define initial conditions
# -------------------------
R0 = [840.5, 485.3, 6905.8]   # Initial position (km)
V0 = [3.7821, -6.5491, 0.0057]  # Initial velocity (km/s)

# -----------------------------------------
# Instantiate the orbit propagator class
# -----------------------------------------
orbit = Orbit_2body()

# -----------------------------------------
# Propagate orbit with J2 perturbation
# Duration: 10 days (864,000 s)
# Time step: 10 seconds
# -----------------------------------------
T_sim_seconds = 24 * 10 * 3600
time_step = 10
r, t = orbit.propagate_with_J2(T=T_sim_seconds, time_step=time_step, R0=R0, V0=V0)

print(f"Propagation complete. {len(t)} steps computed.")


In [ ]:
# ----------------------------
# Visualizations
# ----------------------------
ob = OrbitVisualizer()

# Static 3D orbit view with Earth and borders
ob.EarthStatic(r, title="Orbit with J2 Perturbation (Static View)")

# Animated orbit view (downsampled)
ob.EarthDynamic(r[::50], t[::50], title="Orbit with J2 Perturbation (10 days)")


In [ ]:
# ----------------------------
# Time utilities demonstration
# ----------------------------
current_utc_time = datetime.now(timezone.utc)

print("\n--- Time Conversion Outputs ---")
print(f"Current UTC: {current_utc_time}")
print(f"Julian Date: {orbit.UTC_to_julian(current_utc_time)}")
print(f"YYDDD Format: {orbit.UTC_to_YYDDD(current_utc_time)}")
print(f"Formatted UTC (6 decimals): {orbit.format_utc(current_utc_time, decimals=6)}")
print(f"Formatted UTC (12 decimals): {orbit.format_utc(current_utc_time, decimals=12)}")


In [ ]:
# ----------------------------
# Export ephemeris files
# ----------------------------
positions = r[:, :3]
velocities = r[:, 3:]

scenario_start_epoch = datetime(year=2025, month=5, day=19, hour=8, minute=20, second=0, tzinfo=timezone.utc)

print("\n--- Ephemeris File Generation ---")

# FreeFlyer/STK-compatible ephemeris
ff_output_message = orbit.save_ephermeris_freeflyer(
    r=positions,
    v=velocities,
    t=t,
    scenario_epoch=scenario_start_epoch,
    file_name="j2_orbit_ff"
)
print(ff_output_message)

# SPICE kernel ephemeris
spk_output_message = orbit.save_to_spk(
    r_vectors=positions,
    v_vectors=velocities,
    time=t,
    scenario_epoch=scenario_start_epoch,
    output_file="j2_orbit_spk",
    kernel_list=["naif0012.tls", "pck00010.tpc"],
    kernel_base_dir="./kernels"
)
print(spk_output_message)

print("\nScript finished. Ephemeris files saved: 'j2_orbit_ff.e' and 'j2_orbit_spk.bsp'.")
